<a href="https://colab.research.google.com/github/mikecinnamon/MLearning/blob/main/Notebooks/covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [ML-26] Example - Semantic search of COVID-19 articles

## Importing the data

In [ ]:
import pandas as pd, numpy
path = 'https://raw.githubusercontent.com/mikecinnamon/Data/main/'
df = pd.read_csv(path + 'covid.csv')

In [ ]:
df.info()

In [ ]:
df[['title', 'abstract']].head()

In [ ]:
pd.concat([df['title'].str.len().describe(), df['abstract'].str.len().describe()], axis=1)

## Q1. Embedding model

In [ ]:
pip install cohere

In [ ]:
import cohere
co = cohere.ClientV2(api_key='YOUR_API_KEY')

In [ ]:
model_name = 'embed-english-v3.0'

## Q2. Encoding the query

In [ ]:
query = ['False positive rate in COVID test']

In [ ]:
query_embed = co.embed(texts=query, model=model_name, input_type='search_query')

In [ ]:
query_embed = np.array(query_embed.embeddings[0])
query_embed.shape

## Q3. Encoding the abstracts

In [ ]:
import time

In [ ]:
abstract = df['abstract'].tolist()

In [ ]:
abstract_embed = co.embed(texts=abstract[:2500], model=model_name, input_type='search_document').embeddings

In [ ]:
for i in range(1, 4):
    time.sleep(60)
    new_embed = co.embed(texts=abstract[(2500*i):2500*(i+1)], model=model_name, input_type='search_document').embeddings
    abstract_embed = abstract_embed + new_embed

In [ ]:
df['abstract_embed'] = [np.array(e) for e in abstract_embed]

In [ ]:
df.info()

In [ ]:
df['abstract_embed'][0]

## Q4. Vector search

In [ ]:
df['similarity'] = df['abstract_embed'].apply(lambda x: np.dot(x, query_embed))
df.head()

In [ ]:
search_output = df.sort_values(by='similarity', ascending=False).head(20)

## Q5. Reranking

In [ ]:
model_name = 'rerank-english-v3.0'

In [ ]:
docs = search_output['abstract'].tolist()

In [ ]:
top3 = co.rerank(model=model_name, query=query[0], documents=docs, top_n=3)
top3.results

In [ ]:
selection = [r.index for r in top3.results]
search_output['url'].iloc[selection]